Array traps and optimisation
============================

First let's start 3 workers

In [ ]:
import dask.array as da
from dask.distributed import Client, progress
client = Client(
    processes=False,
    n_workers=3,
    threads_per_worker=2,
)
client

Chunks size
-----------

- Too large chunks don't split work efficiently.
- Too small and too much time is lost in communication and other overheads.
- 100Mb~1Gb per chunks is usually good. Scheduling a single task take arround ~1ms.

In [ ]:
print("1 chunk")
arr = da.random.random((6000, 6000), chunks=(6000, 6000))
%time x = arr.sum().compute()
print("3 chunks")
arr = da.random.random((6000, 6000), chunks=(2000, 6000))
%time x = arr.sum().compute()
print("4 chunks")
arr = da.random.random((6000, 6000), chunks=(3000, 3000))
%time x = arr.sum().compute()
print("36 chunks")
arr = da.random.random((6000, 6000), chunks=(1000, 1000))
%time x = arr.sum().compute()
print("400 chunks")
arr = da.random.random((6000, 6000), chunks=(300, 300))
%time x = arr.sum().compute()
print("auto")
arr = da.random.random((6000, 6000), chunks="auto")
%time x = arr.sum().compute()

In [ ]:
arr

Operation order
---------------

Dask has a symbolic tree of operation, but little tools for optimization.  
It does not reorder operations for faster computation:

In [ ]:
A = da.random.random((3000, 3000), chunks=(1000, 1000))
B = da.random.random((3000, 3000), chunks=(1000, 1000))
v = da.random.random((3000, 1), chunks=(1000, 1))
MM1 = (A @ B) @ v
MM2 = A @ (B @ v)
%time MM1.compute()
%time MM2.compute()

In [ ]:
# einsum can do operation in the right order, but the operation is not optimized
einMM = da.einsum("ij,jk,kl->il", A, B, v)
%time einMM.compute()